In [1]:
# This preprocess scripts was made by https://github.com/Alexander-H-Liu/End-to-end-ASR-Pytorch, I just edited some stuff
%load_ext autoreload
%autoreload 2

from pydub import AudioSegment
import os
import numpy as np
from tqdm import tqdm
from tqdm import notebook
from joblib import Parallel, delayed
import scipy.io.wavfile as wav
from python_speech_features import logfbank
from functions import traverse, wav2logfbank, flac2wav, norm, char_mapping
from functions import file_list_path
import argparse



In [5]:
# parameter setup

#parser = argparse.ArgumentParser(description="Librispeech preprocess.")


#root = 'C:\\Users\\USER\\Desktop\\jiwon\\las-pytorch\\data\\kospeech\\'

# "number of cpu availible for preprocessing.\n -1: use all cpu, -2: use all cpu but one"
n_jobs = -2

#number of filters for bank
n_filters = 40

# [25ms]
win_size = 0.025

# "Normalize features s.t. mean = 0 std = 1"
norm_x = False



In [4]:
# 파일 경로 리스트 가져오기

root = 'C:\\Users\\USER\\Desktop\\jiwon\\las-pytorch\\data\\kospeech'
tr_file_list = file_list_path(root, "train","source",'wav')
dev_file_list = file_list_path(root, "dev","source",'wav')
tt_clean_file_list = file_list_path(root, "test","source",'wav')

In [5]:
print(tt_clean_file_list[:10])

['C:\\Users\\USER\\Desktop\\jiwon\\las-pytorch\\data\\kospeech\\test\\source\\일반남여_일반통합01_F_KKJ00_47_수도권_녹음실\\일반남여_일반통합01_F_KKJ00_47_수도권_녹음실_00001.wav', 'C:\\Users\\USER\\Desktop\\jiwon\\las-pytorch\\data\\kospeech\\test\\source\\일반남여_일반통합01_F_KKJ00_47_수도권_녹음실\\일반남여_일반통합01_F_KKJ00_47_수도권_녹음실_00002.wav', 'C:\\Users\\USER\\Desktop\\jiwon\\las-pytorch\\data\\kospeech\\test\\source\\일반남여_일반통합01_F_KKJ00_47_수도권_녹음실\\일반남여_일반통합01_F_KKJ00_47_수도권_녹음실_00003.wav', 'C:\\Users\\USER\\Desktop\\jiwon\\las-pytorch\\data\\kospeech\\test\\source\\일반남여_일반통합01_F_KKJ00_47_수도권_녹음실\\일반남여_일반통합01_F_KKJ00_47_수도권_녹음실_00004.wav', 'C:\\Users\\USER\\Desktop\\jiwon\\las-pytorch\\data\\kospeech\\test\\source\\일반남여_일반통합01_F_KKJ00_47_수도권_녹음실\\일반남여_일반통합01_F_KKJ00_47_수도권_녹음실_00005.wav', 'C:\\Users\\USER\\Desktop\\jiwon\\las-pytorch\\data\\kospeech\\test\\source\\일반남여_일반통합01_F_KKJ00_47_수도권_녹음실\\일반남여_일반통합01_F_KKJ00_47_수도권_녹음실_00006.wav', 'C:\\Users\\USER\\Desktop\\jiwon\\las-pytorch\\data\\kospeech\\test\\source\\일반남여_일반통합0

In [ ]:
# source 파일 전처리

# PCM -> WAV 파일 전환


import soundfile as sf

for file in tqdm(tr_file_list):

    sample_data = file
    data_dir ='\\'.join(file.split("\\")[:-1])


    signal = np.memmap(sample_data, dtype='h', mode='r').astype('float32')
    signal = signal / 32767

    sf.write(os.path.join(data_dir, file+'.wav'), signal, 16000, format='WAV', endian='LITTLE', subtype='PCM_16')
    os.remove(file)








  0%|                                                                                        | 0/77632 [00:00<?, ?it/s]






  0%|                                                                              | 15/77632 [00:00<08:53, 145.60it/s]






  0%|                                                                              | 30/77632 [00:00<09:26, 136.99it/s]






  0%|                                                                              | 44/77632 [00:00<09:31, 135.80it/s]






  0%|                                                                              | 59/77632 [00:00<09:21, 138.22it/s]






  0%|                                                                               | 73/77632 [00:00<13:47, 93.75it/s]






  0%|                                                                               | 84/77632 [00:00<13:45, 93.92it/s]






  0%|                                                                              | 99/77632 [00:00<12:10, 106

In [23]:
root = 'C:\\Users\\USER\\Desktop\\jiwon\\las-pytorch\\data\\kospeech'
tr_file_list = file_list_path(root, "train","source","wav")
dev_file_list = file_list_path(root, "dev","source","wav")
tt_clean_file_list = file_list_path(root, "test","source","wav")

In [11]:
# WAV 파일을 가지고 npy 파일 생성

print("Training", flush=True)
results = Parallel(n_jobs=n_jobs, backend="threading")(
    delayed(wav2logfbank)(i[:-4] + ".wav", win_size, n_filters) for i in notebook.tqdm(tr_file_list)
)

print("Validation", flush=True)
results = Parallel(n_jobs=n_jobs, backend="threading")(
    delayed(wav2logfbank)(i[:-4] + ".wav", win_size, n_filters) for i in notebook.tqdm(dev_file_list)
)

print("Test clean", flush=True)
results = Parallel(n_jobs=n_jobs, backend="threading")(
    delayed(wav2logfbank)(i[:-4] + ".wav", win_size, n_filters) for i in notebook.tqdm(tt_clean_file_list)
)

Training


  0%|          | 0/77632 [00:00<?, ?it/s]

Validation


  0%|          | 0/19775 [00:00<?, ?it/s]

Test clean


  0%|          | 0/7318 [00:00<?, ?it/s]

In [25]:
print("Training", flush=True)
results = Parallel(n_jobs=n_jobs, backend="threading")(
    delayed(wav2logfbank)(i[:-4] + ".wav", win_size, n_filters) for i in notebook.tqdm(tr_file_list)
)


Training


  0%|          | 0/214300 [00:00<?, ?it/s]

In [10]:
from functions import file_list
root = 'C:\\Users\\USER\\Desktop\\jiwon\\las-pytorch\\data\\kospeech'
data_type = "train"
label_source = "source"
pre_fix = ".fb"
li1 = file_list(root, data_type, label_source,pre_fix)
li2 = file_list(root, data_type, label_source,".wav")
li3 = file_list(root, data_type, "label",".json")
print("num of jb",len(li1))
print("num of wav",len(li2))
print("num of json",len(li3))

li4 = file_list(root, "dev", label_source,pre_fix)
li5 = file_list(root, data_type, label_source,".wav")
li6 = file_list(root, data_type, "label",".json")
print("num of jb",len(li1))
print("num of wav",len(li2))
print("num of json",len(li3))

li7 = file_list(root, "test", label_source,pre_fix)
li8 = file_list(root, data_type, label_source,".wav")
li9 = file_list(root, data_type, "label",".json")
print("num of jb",len(li1))
print("num of wav",len(li2))
print("num of json",len(li3))

print(len(li1) + len(li4) + len(li7))

# label이 더 많다...

num of jb 214300
num of wav 214300
num of json 214300
num of jb 214300
num of wav 214300
num of json 214300
num of jb 214300
num of wav 214300
num of json 214300
241393


In [70]:
fb_file_names = file_list(root, data_type, label_source,".fb")
json_list = file_list_path(root, data_type, "label",".json")

print("num of fb",len(li1))
print("num of json",len(li3))
print(len(li3) - len(li1))

num of fb 214300
num of json 214789
489


In [63]:
fb_file_names_no_extension = []
for name in fb_file_names:
    fb_file_names_no_extension.append(".".join(name.split('.')[:-2]))
print(fb_file_names_no_extension[:3])

['일반남여_일반통합01_F_HSH00_28_수도권_녹음실_00001', '일반남여_일반통합01_F_HSH00_28_수도권_녹음실_00002', '일반남여_일반통합01_F_HSH00_28_수도권_녹음실_00003']


In [69]:
from tqdm.auto import tqdm
import os

json_name = json_list[0]
print(json_name)
no_extension_json = json_name.split("\\")[-1].split(".")[0]
print(no_extension_json in fb_file_names_no_extension)

cnt=0
for json_name in tqdm(json_list):
    no_extension_json = json_name.split("\\")[-1].split(".")[0]
    if no_extension_json not in fb_file_names_no_extension:
        os.remove(json_name)
        cnt+=1
print(cnt)

C:\Users\USER\Desktop\jiwon\las-pytorch\data\kospeech\train\label\일반남여_일반통합01_F_HSH00_28_수도권_녹음실\일반남여_일반통합01_F_HSH00_28_수도권_녹음실_00001.json
True


  0%|          | 0/214789 [00:00<?, ?it/s]

489


In [72]:
# label 파일 정처리
%load_ext autoreload
%autoreload 2
from functions import make_txt_file


root = 'C:\\Users\\USER\\Desktop\\jiwon\\las-pytorch\\data\\kospeech'
data_type = "test"
label_source = "label"


#make_txt_file(root,"test",label_source)
#make_txt_file(root,"dev",label_source)
make_txt_file(root,"train",label_source)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


  0%|          | 0/164 [00:00<?, ?it/s]

In [73]:
tr_file_list = file_list_path(root, "train","source",".fb" + str(n_filters))
tr_text = file_list_path(root, "train", "source",".txt",True)
dev_file_list = file_list_path(root, "dev","source",".fb" + str(n_filters))
dev_text = file_list_path(root, "dev", "source",".txt",True)
test_file_list = file_list_path(root, "test","source",".fb" + str(n_filters))
tt_text = file_list_path(root, "test", "source",".txt",True)

print(len(tr_file_list))
print(len(tr_text))
print("-"*10)
print(len(dev_file_list))
print(len(dev_text))
print("-"*10)
print(len(test_file_list))
print(len(tt_text))

214300
214300
----------
19775
19775
----------
7318
7318


['일반남여_일반통합01_F_HSH00_28_수도권_녹음실_00001.wav', '일반남여_일반통합01_F_HSH00_28_수도권_녹음실_00002.wav', '일반남여_일반통합01_F_HSH00_28_수도권_녹음실_00003.wav']
['일반남여_일반통합01_F_HSH00_28_수도권_녹음실_00001.json', '일반남여_일반통합01_F_HSH00_28_수도권_녹음실_00002.json', '일반남여_일반통합01_F_HSH00_28_수도권_녹음실_00003.json']


In [15]:
%load_ext autoreload
%autoreload 2
from functions import char_mapping
from functions import file_list_path

root = 'C:\\Users\\USER\\Desktop\\jiwon\\las-pytorch\\data\\kospeech'
target_path = root+'\\processed\\'

tr_file_list = file_list_path(root, "train","source",".fb" + str(n_filters))
tr_text = file_list_path(root, "train", "source",".txt",True)
dev_file_list = file_list_path(root, "dev","source",".fb" + str(n_filters))
dev_text = file_list_path(root, "dev", "source",".txt",True)
test_file_list = file_list_path(root, "test","source",".fb" + str(n_filters))
tt_text = file_list_path(root, "test", "source",".txt",True)

total_file_list = tr_file_list + dev_file_list + test_file_list
total_text = tr_text + dev_text + tt_text

char_map = char_mapping(total_text, target_path,seq="first")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
target_path = root+'\\processed\\'


# text to index sequence
tmp_list = []
for text in tr_text:
    tmp = []
    for char in text:
        tmp.append(char_map[char])
    tmp_list.append(tmp)
tr_text = tmp_list

del tmp_list



# text to index sequence
tmp_list = []
for text in dev_text:
    tmp = []
    for char in text:
        tmp.append(char_map[char])
    tmp_list.append(tmp)
dev_text = tmp_list
del tmp_list



print()


NameError: name 'tr_text' is not defined

In [16]:
print("Preparing Test clean Dataset...", flush=True)
root = 'C:\\Users\\USER\\Desktop\\jiwon\\las-pytorch\\data\\kospeech'
test_clean_path = root+'\\test\\source'
target_path = root+'\\processed\\'



tmp_list = []
for text in tt_text:
    tmp = []
    for char in text:
        tmp.append(char_map[char])
    tmp_list.append(tmp)
tt_text = tmp_list
del tmp_list

Preparing Test clean Dataset...


In [75]:
%load_ext autoreload
%autoreload 2
from functions import char_mapping
from functions import file_list_path


# write dataset
file_name = "train.csv"

print("Writing dataset to " + target_path + file_name + "...", flush=True)

with open(target_path + file_name, "w", encoding='utf8') as f:
    f.write("idx,input,label\n")
    for i in range(len(tr_file_list)):
        f.write(str(i) + ",")
        f.write(tr_file_list[i] + ",")
        for char in tr_text[i]:
            f.write(" " + str(char))
        f.write("\n")

print()
print("Preparing Validation Dataset...", flush=True)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Writing dataset to C:\Users\USER\Desktop\jiwon\las-pytorch\data\kospeech\processed\train.csv...

Preparing Validation Dataset...


In [76]:
%load_ext autoreload
%autoreload 2
from functions import char_mapping
from functions import file_list_path



# write dataset
file_name = "dev.csv"

print("Writing dataset to " + target_path + file_name + "...", flush=True)

with open(target_path + file_name, "w", encoding='utf8') as f:
    f.write("idx,input,label\n")
    for i in range(len(dev_file_list)):
        f.write(str(i) + ",")
        f.write(dev_file_list[i] + ",")
        for char in dev_text[i]:
            f.write(" " + str(char))
        f.write("\n")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Writing dataset to C:\Users\USER\Desktop\jiwon\las-pytorch\data\kospeech\processed\dev.csv...


In [17]:

root = 'C:\\Users\\USER\\Desktop\\jiwon\\las-pytorch\\data\\kospeech'
target_path = root+'\\processed\\'


# write dataset
file_name = "test-clean.csv"
#tt_text = file_list_path(root, "test", "source",".txt",True)
#test_file_list = file_list_path(root, "test","source",".fb" + str(n_filters))
print("Writing dataset to " + target_path + file_name + "...", flush=True)

with open(target_path + file_name, "w", encoding='utf8') as f:
    f.write("idx,input,label\n")
    for i in range(len(test_file_list)):
        f.write(str(i) + ",")
        f.write(test_file_list[i] + ",")
        for char in tt_text[i]:
            f.write(" " + str(char))
        f.write("\n")



Writing dataset to C:\Users\USER\Desktop\jiwon\las-pytorch\data\kospeech\processed\test-clean.csv...
